<a href="https://colab.research.google.com/github/kevinrchilders/computational-number-theory/blob/master/cryptography_chapter_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from collections import Counter
import random, math

In [ ]:
# Fast powering algorithm, gcd algorithms, etc.

def binary(n):
  binary_repn = []
  if n > 1:
    binary_repn = binary(n // 2)
  binary_repn.append(n % 2)
  return binary_repn

def power(g, A, N):
  A = binary(A)
  total=1
  for i in range(len(A)):
    if A[len(A)-i-1]:
      total = (total*g) % N
    g = (g*g) % N
  return total

def gcd(a, b):
  return a if b==0 else gcd(b, a%b)

def extended_gcd(a, b):
  u, g, x, y = 1, a, 0, b
  while y != 0:
    q, t = g // y, g % y
    s = u - q*x
    u, g = x, y
    x, y = s, t
  v = (g - a*u) // b 
  return g, u, v

def order(a, p):
  n = 1
  x = a
  while power(a, n, p) != 1:
    x = (x * a) % p
    n += 1
  return n

def is_primitive(a, p):
  return order(a, p) == p-1

def is_mrprime(n, trials=50):
  for i in range(trials):
    a = random.randint(1, n)
    if is_mrwitness(a, n):
      return False
  return True

def generate_prime(digits, attempts=100):
  N = 2 * 3 * 5 * 7
  for K in range(int(10**(digits)/N), int(10**(digits)/N)+attempts):
    if is_mrprime(N*K + 1):
      return N*K + 1
  print('No primes found. Try more attempts.')
  return None

def is_mrwitness(a, n):
  # If a and n have a common factor, then n is composite
  if gcd(a, n) != 1:
    return True

  # Write n-1 = 2^k*q with q odd
  k=0
  q=n-1
  while q%2 == 0:
    k += 1
    q = q//2
  
  # If a^q == 1 (mod n) then a is not a Miller-Rabin witness for n
  a = power(a, q, n)
  if a == 1:
    return False
  
  # If a^(2^iq) == -1 (mod n) for some i<k then a is not a Miller-Rabin witness for n
  for i in range(k):
    if a == n-1:
      return False
    a = power(a, 2, n)

  return True # Otherwise a is a Miller-Rabin witness for n

def pollard(N, a=2, maxn=1000000):
  for j in range(2, maxn):
    a = power(a, j, N)
    d = gcd(a-1, N)
    if d != 1 and d != N:
      return d
  print('Test failed, try a larger maxn.')
  return None

def find_primitive(p):
  a = 2
  while not is_primitive(a, p):
    a += 1
  return a

# Vigenere cipher

In [ ]:
# We will attempt to decrypt the following message

message = 'zpgdl rjlaj kpylx zpyyg lrjgd lrzhz qyjzq repvm swrzy rigzh zvreg kwivs saolt nliuw oldie aqewf iiykh bjowr hdogc qhkwa jyagg emisr zqoqh oavlk bjofr ylvps rtgiu avmsw lzgms evwpc dmjsv jqbrn klpcf iowhv kxjbj pmfkr qthtk ozrgq ihbmq sbivd ardym qmpbu nivxm tzwqv gefjh ucbor vwpcd xuwft qmoow jipds fluqm oeavl jgqea lrkti wvext vkrrg xani'
message = message.replace(' ', '')
message

In [ ]:
# An analysis of the letter frequencies

counter = Counter(message)
counter.most_common()
print('total number of characters:', len(counter.most_common()))
print('letter and percent frequency:')
for (ch, i) in counter.most_common():
  print(ch, round(100*i/len(message), 1))

In [ ]:
# An analysis of letter frequencies for a block of Shakespeare text

shake = 'From Fife, great king; Where the Norweyan banners flout the sky And fan our people cold. Norway himself, With terrible numbers, Assisted by that most disloyal traitor The thane of Cawdor, began a dismal conflict; Till that Bellonas bridegroom, lappd in proof, Confronted him with self-comparisons, Point against point rebellious, arm gainst arm. Curbing his lavish spirit: and, to conclude, The victory fell on us. Doubtful it stood;As two spent swimmers, that do cling togetherAnd choke their art. The merciless Macdonwald--Worthy to be a rebel, for to thatThe multiplying villanies of natureDo swarm upon him--from the western islesOf kerns and gallowglasses is supplied;And fortune, on his damned quarrel smiling,Showd like a rebels whore: but alls too weak:For brave Macbeth--well he deserves that name--Disdaining fortune, with his brandishd steel,Which smoked with bloody execution,Like valours minion carved out his passageTill he faced the slave;Which neer shook hands, nor bade farewell to him,Till he unseamd him from the nave to the chaps,And fixd his head upon our battlements.'
shake = shake.lower()
shake = shake.replace(',', '')
shake = shake.replace('.', '')
shake = shake.replace('-', '')
shake = shake.replace(';', '')
shake = shake.replace(':', '')
shake = shake.replace(' ', '')
counter = Counter(shake)
counter.most_common()
print('total number of characters:', len(counter.most_common()))
print('letter and percent frequency:')
for (ch, i) in counter.most_common():
  print(ch, round(100*i/len(message), 1))

In [ ]:
# A function for computing the index of coincidence (the probability that two randomly chosen characters will match)
# For a random string ind_co(s) ~ 0.0385
# For the English language ind_co(s) ~ 0.0685

def ind_co(s):
  counter = Counter(s)
  frequencies = np.array(list(counter.values()))
  n = len(s)
  return np.sum(frequencies * (frequencies -1)) / n / (n-1)

In [ ]:
ind_co(message)

In [ ]:
ind_co(shake)

In [ ]:
# An attempt to guess the length of the keyword used for a Vigenere cipher using index of coincidence

for k in range(3, 10):
  print('k=', k, '---------------')
  for i in range(k):
    print(ind_co(message[i::k]))

In [ ]:
k = 

In [ ]:
def shift(s, sigma):
  alphabet = 'abcdefghijklmnopqrstuvwxyz'
  t=''
  for i in range(len(s)):
    t = t + alphabet[(alphabet.find(s[i])+sigma) % 26]
  return t

def mult_ind_co(s, t):
  cs = Counter(s)
  n = len(s)
  ct = Counter(t)
  m = len(t)
  return np.sum([cs[key] * ct[key] for key in (cs+ct).keys()]) / n / m

In [ ]:
mult_ind_co(message, shake)

In [ ]:
# Use mult_ind_co to find probable shifts differences between strings si=message[i::k] as i varies

for i in range(k-1):
  print('i =', i, '----------------------')
  for j in range(i+1,k):
    for sigma in range(26):
      mic = mult_ind_co(shift(message[i::k], sigma), message[j::k])
      if mic > 0.06:
        print('j =', j, ', shift =', sigma, ':', mic)

In [ ]:
# Attempt to find the keyword using most probable shifts

alp = 'abcdefghijklmnopqrstuvwxyz'

base = alp[] + alp[] + alp[] + alp[] + alp[] + alp[] + alp[]
for sigma in range(26):
  print(shift(base, sigma))

In [ ]:
# functions to encrypt/decrypt using a Vigenere cipher

def vig_encrypt(message, keyword):
  alphabet = 'abcdefghijklmnopqrstuvwxyz'
  n = len(message)
  k = len(keyword)
  c = ''
  for i in range(n):
    c = c + alphabet[(alphabet.find(message[i]) + alphabet.find(keyword[i%k])) % 26]
  return c

def vig_decrypt(message, keyword):
  alphabet = 'abcdefghijklmnopqrstuvwxyz'
  n = len(message)
  k = len(keyword)
  c = ''
  for i in range(n):
    c = c + alphabet[(alphabet.find(message[i]) - alphabet.find(keyword[i%k])) % 26]
  return c

In [ ]:
vig_decrypt(message, )

# Collision argument

In [ ]:
# A probabilistic collision algorithm for the discrete log problem g^x = h mod p, where g has order ord

def collision_dlp(g, h, p, ord=None):
  if ord==None:
    ord = p-1

  Y = [random.randint(1,ord) for _ in range(int(3*math.sqrt(ord)))]  # Random exponents y
  gY = [power(g, y, p) for y in Y]                                  # g^y
  Z = [random.randint(1,ord) for _ in range(int(3*math.sqrt(ord)))]  # Random exponents z
  gZh = [(power(g, z, p)*h)%p for z in Z]                           # g^z*h

  # Find a common element of gY and gZh
  A = sorted(list(set(gY))+list(set(gZh)))
  i=0
  while A[i] != A[i+1]:
    i += 1
  common = A[i]

  # Find g^y = g^z*h
  i=0
  while gY[i] != common:
    i += 1
  j=0
  while gZh[j] != common:
    j += 1

  return (Y[i] - Z[j])%(ord)  # Return y-z

### 10-digit example

In [ ]:
# Here's an example with a 10 digit prime.
p = 3267000013
g = 2

In [ ]:
(p-1) == (2*2*3*272250001)

In [ ]:
is_mrprime(272250001)

In [ ]:
# We can see that g has order 4*q = (p-1)/3
q = 272250001
S = [1, 2, 3, 4, 6, 12, q, 2*q, 3*q, 4*q, 6*q, 12*q]
for s in S:
  print(s, power(g, s, p))

In [ ]:
# Let's take a random h so that g^x = h (mod p) has a solution
h = power(g, random.randint(1, 4*q), p)
h

In [ ]:
%%time
x = collision_dlp(g, h, p, 4*q)
x

In [ ]:
# Double check our solution
power(g, x, p) == h

### 15-digit example

In [ ]:
p = 399899999999999


In [ ]:
(p-1) == (2*61*6029*543683671)

In [ ]:
is_mrprime(543683671), is_mrprime(6029)

In [ ]:
g = 7
print(power(g, (p-1)//2, p))
print(power(g, (p-1)//61, p))
print(power(g, (p-1)//6029, p))
print(power(g, (p-1)//543683671, p))

In [ ]:
h = power(g, random.randint(1,(p-1)), p)
h

In [ ]:
%%time
x = collision_dlp(g, h, p)
x